# Note

The code in this notebook is a COPY of the Google Colab notebook, which is where the project was completed. The code here does not run.

Visit https://drive.google.com/drive/folders/11WEvziEDHmUAutBqZiCRYHX5vwRg5zOO?usp=sharing for Google Colab.

# Dependencies

In [ ]:
# import our dependencies
!pip install keras-tuner pyyaml h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Data Preprocessing

In [ ]:
#  import and read the charity_data.csv
url = 'https://raw.githubusercontent.com/cdenq/charity-funding-success-predictor/main/Main/Resources/charity_data.csv'
df = pd.read_csv(url)
df_len = len(df.index)
print(df_len)
df.head()

In [3]:
# drop the non-beneficial ID columns, 'EIN' and 'NAME'
df.drop(['EIN', 'NAME'], axis=1, inplace=True)
df.head()

In [4]:
# determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_values_s = df['APPLICATION_TYPE'].value_counts()
app_values_s

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Compile smaller vars into "other"
application_types_to_replace = []
for i, v in app_values_s.iteritems():
  if v < app_values_s[7]: #anything outside of top 7 is grouped into other
    application_types_to_replace.append(i)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Look at CLASSIFICATION value counts for binning
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1370        1
C4120        1
C2600        1
C2190        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [25]:
# looking at CLASSIFICATION value counts >1
bool_s = df['CLASSIFICATION'].value_counts()>1
class_value_s = df['CLASSIFICATION'].value_counts()[bool_s]
class_value_s

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C1234        2
C1246        2
C1267        2
C3200        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# dropping rows where classification < 2
class_1 = df['CLASSIFICATION'].value_counts()[df['CLASSIFICATION'].value_counts()<2].index.to_list()
df = df[df['CLASSIFICATION'].isin(class_1) == False]
print(f'Was {df_len}, dropped {df_len - len(df.index)}, now {len(df.index)}')

In [27]:
# Assign cutoff value and create a list of classifications to be replaced
classifications_to_replace = []
for i, v in class_value_s.iteritems():
  if v < class_value_s[7]: #anything outside of top 7 is grouped into other
    classifications_to_replace.append(i)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [28]:
# convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(X)
X_dummies.head()

In [31]:
# Split our preprocessed data into our features and target arrays
X = df_dummies.drop('IS_SUCCESSFUL', axis=1)
y = df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42) #because 42 is the only random state we live by

In [32]:
# Create a StandardScaler instances
#assuming data is normal, hence use standardscalar
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

## Compile, Train and Evaluate the Model

In [33]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Import the kerastuner library
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [37]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs = 100, validation_data=(X_test_scaled,y_test))

8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391


In [38]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
best_model.save('AlphabetSoupCharity_Optimized.h5')